In [27]:
#Rohan Robinson
#data from Kaggle: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
#Implementing a Perceptron to classify Kaggle Breast Cancer data as either Benign or Malignant
#building a perceptron class that learns with back propagation

#importing neccesary tools and data pre-processing
import math
import numpy as np
import pandas as pd
import random

#convert kaggle csv data into a dataFrame
cells_df = pd.read_csv("kaggle_breast_cancer_cell_data.csv")

#removing 20 features not relevant 
cells_df = cells_df.drop("radius_se", axis=1)
cells_df = cells_df.drop("texture_se", axis=1)
cells_df = cells_df.drop("perimeter_se", axis=1)
cells_df = cells_df.drop("area_se", axis=1)
cells_df = cells_df.drop("smoothness_se", axis=1)
cells_df = cells_df.drop("compactness_se", axis=1)
cells_df = cells_df.drop("concavity_se", axis=1)
cells_df = cells_df.drop("concave points_se", axis=1)
cells_df = cells_df.drop("symmetry_se", axis=1)
cells_df = cells_df.drop("fractal_dimension_se", axis=1)
cells_df = cells_df.drop("radius_worst", axis=1)
cells_df = cells_df.drop("texture_worst", axis=1)
cells_df = cells_df.drop(["perimeter_worst", "area_worst","smoothness_worst","compactness_worst"], axis=1)
cells_df = cells_df.drop(["concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst"], axis=1)


labels_column = cells_df['diagnosis']

labels = []
m_count = 0
# iterate over all the labels
for label in labels_column.iteritems(): 
    cell_type = label[1]
    
    if cell_type == "M":
        labels.append(1)
        m_count += 1
    elif cell_type == "B":
        labels.append(0)

#further processing
cells_df = cells_df.drop(["id", "diagnosis"], axis=1)


#transforms dataframe into a a 2D list where each index is an instance
cells_collection = []
for row in cells_df.itertuples():
    row_list = []
    for i in range(1,len(row)):
        row_list.append(row[i])
        
    cells_collection.append(row_list)

#end of data preprocessing




In [28]:
 #perceptron implementation

class Perceptron(object):

    def __init__(self, num_features, num_iterations, learning_rate, threshold):
        
        self.learning_rate = learning_rate
        self.weights = np.zeros(num_features + 1)
        self.num_iterations = num_iterations
        self.num_features = num_features
        self.threshold = threshold
        
    def predict_linear(self, feature_vector): #binary linear step activation function 
        
        #convert feature vector into a numpy array
        fv_array = np.asarray(feature_vector)
        
        #weighted sum
        summation = np.dot(fv_array, self.weights[1:]) + self.weights[0]
        
        #activation function logic
        if summation > 0:
            output = 1 #classified as malignant
        else:
            output = 0 #classified as benign
        
        return output
    
    def predict_sigmoid(self, feature_vector): #logistic sigmoid activation function
        
        #convert feature vector into a numpy array
        fv_array = np.asarray(feature_vector)
        
        #weighted sum
        summation = np.dot(fv_array, self.weights[1:]) + self.weights[0]
        
        #activation function logic
        if summation < 0:
            sigmoid_value =  1 - 1/(1 + math.exp(summation))
        else:
            sigmoid_value =  1/(1 + math.exp(-summation))
        
        if sigmoid_value < self.threshold:
            return 0
        
        elif sigmoid_value >= self.threshold:
            return 1
    
    def predict_tanh(self, feature_vector): #hyperbolic tangent activation function
        
        #convert feature vector into a numpy array
        fv_array = np.asarray(feature_vector)
        
        #weighted sum
        summation = np.dot(fv_array, self.weights[1:]) + self.weights[0]
        summation = 2 * summation
        #activation function logic
        if summation < 0:
            tanh_value = 2 * (1 - 1/(1 + math.exp(summation))) - 1
        else:
            tanh_value = 2 * (1/(1 + math.exp(-summation))) - 1
        
        if tanh_value < 0:
            return 0
        
        elif tanh_value >= 0:
            return 1
        
        
    def learn_weights_tanh(self, features, labels):
        
        for i in range(self.num_iterations):
            print("iteration", i)
            error_total = 0
            for feature_vector, label in zip(features, labels):

                prediction = self.predict_tanh(feature_vector)
                error = label - prediction
                error_total += abs(error)
                
                self.weights[0] += self.learning_rate * error
                
                for i in range(1, self.num_features + 1):
                    self.weights[i] += self.learning_rate * error * feature_vector[i-1]
                    
            accuracy = 1 - (error_total/len(labels))   
            print("accuracy", accuracy)
                    
        return self.weights
    
    def learn_weights_sigmoid(self, features, labels):
        
         for i in range(self.num_iterations):
            print("iteration", i)
            error_total = 0
            for feature_vector, label in zip(features, labels):

                prediction = self.predict_sigmoid(feature_vector)
                error = label - prediction
                error_total += abs(error)
                
                self.weights[0] += self.learning_rate * error
                
                for i in range(1, self.num_features + 1):
                    self.weights[i] += self.learning_rate * error * feature_vector[i-1]
                    
            accuracy = 1 - (error_total/len(labels))
            print("accuracy", accuracy)        
                    
            
                    
         return self.weights
    
    def learn_weights_linear(self, features, labels):
        
         for i in range(self.num_iterations):
            print("iteration", i)
            error_total = 0
            for feature_vector, label in zip(features, labels):

                prediction = self.predict_tanh(feature_vector)
                error = label - prediction
                error_total += abs(error)
                    
                self.weights[0] += self.learning_rate * error
            
                for i in range(1, self.num_features + 1):
                    self.weights[i] += self.learning_rate * error * feature_vector[i-1]
            accuracy = 1 - (error_total/len(labels))  
            print("accuracy", accuracy)

         return self.weights
            
        



In [29]:
#test for linear function activated perceptron
#--------------------------------------
num_features = len(cells_collection[1])
#create a perceptron
linear_p = Perceptron(num_features, num_iterations = 5000, learning_rate = 0.005, threshold = 0.5)
final_linear_weights = linear_p.learn_weights_linear(cells_collection, labels)
print("final linear-activated weights", final_linear_weights)


iteration 0
accuracy 0.6889279437609842
iteration 1
accuracy 0.70298769771529
iteration 2
accuracy 0.718804920913884
iteration 3
accuracy 0.7750439367311073
iteration 4
accuracy 0.7680140597539543
iteration 5
accuracy 0.7855887521968365
iteration 6
accuracy 0.7873462214411248
iteration 7
accuracy 0.7996485061511424
iteration 8
accuracy 0.8137082601054482
iteration 9
accuracy 0.8260105448154658
iteration 10
accuracy 0.8154657293497364
iteration 11
accuracy 0.820738137082601
iteration 12
accuracy 0.8066783831282953
iteration 13
accuracy 0.8347978910369068
iteration 14
accuracy 0.8260105448154658
iteration 15
accuracy 0.8330404217926186
iteration 16
accuracy 0.8295254833040422
iteration 17
accuracy 0.8400702987697715
iteration 18
accuracy 0.8242530755711774
iteration 19
accuracy 0.836555360281195
iteration 20
accuracy 0.8330404217926186
iteration 21
accuracy 0.8400702987697715
iteration 22
accuracy 0.8400702987697715
iteration 23
accuracy 0.8400702987697715
iteration 24
accuracy 0.8312829

accuracy 0.8664323374340949
iteration 238
accuracy 0.8629173989455184
iteration 239
accuracy 0.8611599297012302
iteration 240
accuracy 0.8576449912126538
iteration 241
accuracy 0.8681898066783831
iteration 242
accuracy 0.8629173989455184
iteration 243
accuracy 0.8611599297012302
iteration 244
accuracy 0.8576449912126538
iteration 245
accuracy 0.8611599297012302
iteration 246
accuracy 0.8576449912126538
iteration 247
accuracy 0.8681898066783831
iteration 248
accuracy 0.8629173989455184
iteration 249
accuracy 0.8611599297012302
iteration 250
accuracy 0.8576449912126538
iteration 251
accuracy 0.8681898066783831
iteration 252
accuracy 0.8629173989455184
iteration 253
accuracy 0.8611599297012302
iteration 254
accuracy 0.8576449912126538
iteration 255
accuracy 0.8681898066783831
iteration 256
accuracy 0.8629173989455184
iteration 257
accuracy 0.8611599297012302
iteration 258
accuracy 0.8576449912126538
iteration 259
accuracy 0.8611599297012302
iteration 260
accuracy 0.8576449912126538
iterat

accuracy 0.8611599297012302
iteration 456
accuracy 0.8734622144112478
iteration 457
accuracy 0.8664323374340949
iteration 458
accuracy 0.8664323374340949
iteration 459
accuracy 0.8611599297012302
iteration 460
accuracy 0.8611599297012302
iteration 461
accuracy 0.8769771528998243
iteration 462
accuracy 0.8576449912126538
iteration 463
accuracy 0.8646748681898067
iteration 464
accuracy 0.8664323374340949
iteration 465
accuracy 0.8681898066783831
iteration 466
accuracy 0.8681898066783831
iteration 467
accuracy 0.8681898066783831
iteration 468
accuracy 0.8681898066783831
iteration 469
accuracy 0.8681898066783831
iteration 470
accuracy 0.8681898066783831
iteration 471
accuracy 0.8681898066783831
iteration 472
accuracy 0.8681898066783831
iteration 473
accuracy 0.8681898066783831
iteration 474
accuracy 0.8681898066783831
iteration 475
accuracy 0.8681898066783831
iteration 476
accuracy 0.8681898066783831
iteration 477
accuracy 0.8681898066783831
iteration 478
accuracy 0.8681898066783831
iterat

accuracy 0.8717047451669596
iteration 657
accuracy 0.8681898066783831
iteration 658
accuracy 0.8629173989455184
iteration 659
accuracy 0.8681898066783831
iteration 660
accuracy 0.8576449912126538
iteration 661
accuracy 0.8717047451669596
iteration 662
accuracy 0.8681898066783831
iteration 663
accuracy 0.8576449912126538
iteration 664
accuracy 0.8717047451669596
iteration 665
accuracy 0.8681898066783831
iteration 666
accuracy 0.8576449912126538
iteration 667
accuracy 0.8717047451669596
iteration 668
accuracy 0.8681898066783831
iteration 669
accuracy 0.8576449912126538
iteration 670
accuracy 0.8717047451669596
iteration 671
accuracy 0.8681898066783831
iteration 672
accuracy 0.8629173989455184
iteration 673
accuracy 0.8681898066783831
iteration 674
accuracy 0.8681898066783831
iteration 675
accuracy 0.8611599297012302
iteration 676
accuracy 0.8629173989455184
iteration 677
accuracy 0.8681898066783831
iteration 678
accuracy 0.8681898066783831
iteration 679
accuracy 0.8681898066783831
iterat

accuracy 0.859402460456942
iteration 877
accuracy 0.859402460456942
iteration 878
accuracy 0.8646748681898067
iteration 879
accuracy 0.859402460456942
iteration 880
accuracy 0.8717047451669596
iteration 881
accuracy 0.859402460456942
iteration 882
accuracy 0.859402460456942
iteration 883
accuracy 0.8646748681898067
iteration 884
accuracy 0.859402460456942
iteration 885
accuracy 0.8717047451669596
iteration 886
accuracy 0.859402460456942
iteration 887
accuracy 0.859402460456942
iteration 888
accuracy 0.8646748681898067
iteration 889
accuracy 0.859402460456942
iteration 890
accuracy 0.859402460456942
iteration 891
accuracy 0.8681898066783831
iteration 892
accuracy 0.8506151142355008
iteration 893
accuracy 0.8629173989455184
iteration 894
accuracy 0.8717047451669596
iteration 895
accuracy 0.859402460456942
iteration 896
accuracy 0.859402460456942
iteration 897
accuracy 0.8646748681898067
iteration 898
accuracy 0.859402460456942
iteration 899
accuracy 0.859402460456942
iteration 900
accura

...

accuracy 0.8629173989455184
iteration 3874
accuracy 0.8611599297012302
iteration 3875
accuracy 0.8681898066783831
iteration 3876
accuracy 0.875219683655536
iteration 3877
accuracy 0.8681898066783831
iteration 3878
accuracy 0.8681898066783831
iteration 3879
accuracy 0.8629173989455184
iteration 3880
accuracy 0.8611599297012302
iteration 3881
accuracy 0.8681898066783831
iteration 3882
accuracy 0.8699472759226714
iteration 3883
accuracy 0.8611599297012302
iteration 3884
accuracy 0.875219683655536
iteration 3885
accuracy 0.8681898066783831
iteration 3886
accuracy 0.8681898066783831
iteration 3887
accuracy 0.8629173989455184
iteration 3888
accuracy 0.8611599297012302
iteration 3889
accuracy 0.8681898066783831
iteration 3890
accuracy 0.8629173989455184
iteration 3891
accuracy 0.8611599297012302
iteration 3892
accuracy 0.8681898066783831
iteration 3893
accuracy 0.8699472759226714
iteration 3894
accuracy 0.8611599297012302
iteration 3895
accuracy 0.875219683655536
iteration 3896
accuracy 0.868

iteration 4157
accuracy 0.8664323374340949
iteration 4158
accuracy 0.8664323374340949
iteration 4159
accuracy 0.8664323374340949
iteration 4160
accuracy 0.8664323374340949
iteration 4161
accuracy 0.8646748681898067
iteration 4162
accuracy 0.8681898066783831
iteration 4163
accuracy 0.8734622144112478
iteration 4164
accuracy 0.8734622144112478
iteration 4165
accuracy 0.8734622144112478
iteration 4166
accuracy 0.8734622144112478
iteration 4167
accuracy 0.8699472759226714
iteration 4168
accuracy 0.8664323374340949
iteration 4169
accuracy 0.8664323374340949
iteration 4170
accuracy 0.8664323374340949
iteration 4171
accuracy 0.8664323374340949
iteration 4172
accuracy 0.8664323374340949
iteration 4173
accuracy 0.8664323374340949
iteration 4174
accuracy 0.8664323374340949
iteration 4175
accuracy 0.8664323374340949
iteration 4176
accuracy 0.8664323374340949
iteration 4177
accuracy 0.8664323374340949
iteration 4178
accuracy 0.8699472759226714
iteration 4179
accuracy 0.8734622144112478
iteration 4

accuracy 0.8664323374340949
iteration 4407
accuracy 0.8664323374340949
iteration 4408
accuracy 0.8664323374340949
iteration 4409
accuracy 0.8664323374340949
iteration 4410
accuracy 0.8664323374340949
iteration 4411
accuracy 0.8629173989455184
iteration 4412
accuracy 0.8664323374340949
iteration 4413
accuracy 0.8664323374340949
iteration 4414
accuracy 0.8664323374340949
iteration 4415
accuracy 0.8664323374340949
iteration 4416
accuracy 0.8664323374340949
iteration 4417
accuracy 0.8664323374340949
iteration 4418
accuracy 0.8664323374340949
iteration 4419
accuracy 0.8699472759226714
iteration 4420
accuracy 0.8629173989455184
iteration 4421
accuracy 0.8629173989455184
iteration 4422
accuracy 0.8664323374340949
iteration 4423
accuracy 0.8664323374340949
iteration 4424
accuracy 0.8664323374340949
iteration 4425
accuracy 0.8664323374340949
iteration 4426
accuracy 0.8699472759226714
iteration 4427
accuracy 0.8734622144112478
iteration 4428
accuracy 0.8699472759226714
iteration 4429
accuracy 0.

iteration 4656
accuracy 0.8664323374340949
iteration 4657
accuracy 0.8664323374340949
iteration 4658
accuracy 0.8664323374340949
iteration 4659
accuracy 0.8664323374340949
iteration 4660
accuracy 0.8664323374340949
iteration 4661
accuracy 0.8576449912126538
iteration 4662
accuracy 0.8611599297012302
iteration 4663
accuracy 0.8611599297012302
iteration 4664
accuracy 0.8611599297012302
iteration 4665
accuracy 0.8699472759226714
iteration 4666
accuracy 0.8664323374340949
iteration 4667
accuracy 0.8664323374340949
iteration 4668
accuracy 0.8664323374340949
iteration 4669
accuracy 0.8664323374340949
iteration 4670
accuracy 0.8664323374340949
iteration 4671
accuracy 0.8664323374340949
iteration 4672
accuracy 0.8664323374340949
iteration 4673
accuracy 0.8734622144112478
iteration 4674
accuracy 0.8576449912126538
iteration 4675
accuracy 0.8699472759226714
iteration 4676
accuracy 0.8699472759226714
iteration 4677
accuracy 0.8734622144112478
iteration 4678
accuracy 0.8699472759226714
iteration 4

accuracy 0.8681898066783831
iteration 4904
accuracy 0.8629173989455184
iteration 4905
accuracy 0.859402460456942
iteration 4906
accuracy 0.8681898066783831
iteration 4907
accuracy 0.859402460456942
iteration 4908
accuracy 0.8699472759226714
iteration 4909
accuracy 0.8769771528998243
iteration 4910
accuracy 0.8646748681898067
iteration 4911
accuracy 0.8699472759226714
iteration 4912
accuracy 0.8717047451669596
iteration 4913
accuracy 0.8681898066783831
iteration 4914
accuracy 0.8629173989455184
iteration 4915
accuracy 0.8717047451669596
iteration 4916
accuracy 0.8629173989455184
iteration 4917
accuracy 0.8717047451669596
iteration 4918
accuracy 0.8629173989455184
iteration 4919
accuracy 0.8717047451669596
iteration 4920
accuracy 0.8629173989455184
iteration 4921
accuracy 0.8629173989455184
iteration 4922
accuracy 0.8769771528998243
iteration 4923
accuracy 0.859402460456942
iteration 4924
accuracy 0.8699472759226714
iteration 4925
accuracy 0.875219683655536
iteration 4926
accuracy 0.8717

In [30]:
#test for sigmoid function activated perceptron
#--------------------------------------
num_features = len(cells_collection[1])
#create a perceptron
sigmoid_p = Perceptron(num_features, num_iterations = 5000, learning_rate = 0.005, threshold = 0.5)
final_sigmoid_weights = sigmoid_p.learn_weights_sigmoid(cells_collection, labels)
print("final sigmoid-activated weights", final_sigmoid_weights)

iteration 0
accuracy 0.6889279437609842
iteration 1
accuracy 0.70298769771529
iteration 2
accuracy 0.718804920913884
iteration 3
accuracy 0.7750439367311073
iteration 4
accuracy 0.7680140597539543
iteration 5
accuracy 0.7855887521968365
iteration 6
accuracy 0.7873462214411248
iteration 7
accuracy 0.7996485061511424
iteration 8
accuracy 0.8137082601054482
iteration 9
accuracy 0.8260105448154658
iteration 10
accuracy 0.8154657293497364
iteration 11
accuracy 0.820738137082601
iteration 12
accuracy 0.8066783831282953
iteration 13
accuracy 0.8347978910369068
iteration 14
accuracy 0.8260105448154658
iteration 15
accuracy 0.8330404217926186
iteration 16
accuracy 0.8295254833040422
iteration 17
accuracy 0.8400702987697715
iteration 18
accuracy 0.8242530755711774
iteration 19
accuracy 0.836555360281195
iteration 20
accuracy 0.8330404217926186
iteration 21
accuracy 0.8400702987697715
iteration 22
accuracy 0.8400702987697715
iteration 23
accuracy 0.8400702987697715
iteration 24
accuracy 0.8312829

accuracy 0.8681898066783831
iteration 279
accuracy 0.8629173989455184
iteration 280
accuracy 0.8681898066783831
iteration 281
accuracy 0.8629173989455184
iteration 282
accuracy 0.8681898066783831
iteration 283
accuracy 0.8629173989455184
iteration 284
accuracy 0.8681898066783831
iteration 285
accuracy 0.8664323374340949
iteration 286
accuracy 0.8646748681898067
iteration 287
accuracy 0.8681898066783831
iteration 288
accuracy 0.8664323374340949
iteration 289
accuracy 0.8611599297012302
iteration 290
accuracy 0.8471001757469244
iteration 291
accuracy 0.8646748681898067
iteration 292
accuracy 0.8681898066783831
iteration 293
accuracy 0.8646748681898067
iteration 294
accuracy 0.8629173989455184
iteration 295
accuracy 0.8681898066783831
iteration 296
accuracy 0.8646748681898067
iteration 297
accuracy 0.8646748681898067
iteration 298
accuracy 0.8646748681898067
iteration 299
accuracy 0.8646748681898067
iteration 300
accuracy 0.8629173989455184
iteration 301
accuracy 0.8664323374340949
iterat

accuracy 0.8681898066783831
iteration 513
accuracy 0.8681898066783831
iteration 514
accuracy 0.8681898066783831
iteration 515
accuracy 0.8681898066783831
iteration 516
accuracy 0.8681898066783831
iteration 517
accuracy 0.8681898066783831
iteration 518
accuracy 0.8681898066783831
iteration 519
accuracy 0.8681898066783831
iteration 520
accuracy 0.8681898066783831
iteration 521
accuracy 0.8681898066783831
iteration 522
accuracy 0.8681898066783831
iteration 523
accuracy 0.8681898066783831
iteration 524
accuracy 0.8681898066783831
iteration 525
accuracy 0.8681898066783831
iteration 526
accuracy 0.8681898066783831
iteration 527
accuracy 0.8717047451669596
iteration 528
accuracy 0.8664323374340949
iteration 529
accuracy 0.8699472759226714
iteration 530
accuracy 0.8664323374340949
iteration 531
accuracy 0.8646748681898067
iteration 532
accuracy 0.8681898066783831
iteration 533
accuracy 0.8699472759226714
iteration 534
accuracy 0.8681898066783831
iteration 535
accuracy 0.8664323374340949
iterat

iteration 778
accuracy 0.8629173989455184
iteration 779
accuracy 0.8646748681898067
iteration 780
accuracy 0.8629173989455184
iteration 781
accuracy 0.8629173989455184
iteration 782
accuracy 0.8629173989455184
iteration 783
accuracy 0.8646748681898067
iteration 784
accuracy 0.8629173989455184
iteration 785
accuracy 0.8629173989455184
iteration 786
accuracy 0.8629173989455184
iteration 787
accuracy 0.8646748681898067
iteration 788
accuracy 0.8629173989455184
iteration 789
accuracy 0.8629173989455184
iteration 790
accuracy 0.8629173989455184
iteration 791
accuracy 0.8646748681898067
iteration 792
accuracy 0.8646748681898067
iteration 793
accuracy 0.859402460456942
iteration 794
accuracy 0.8506151142355008
iteration 795
accuracy 0.8629173989455184
iteration 796
accuracy 0.859402460456942
iteration 797
accuracy 0.8506151142355008
iteration 798
accuracy 0.8629173989455184
iteration 799
accuracy 0.859402460456942
iteration 800
accuracy 0.8629173989455184
iteration 801
accuracy 0.864674868189

accuracy 0.8506151142355008
iteration 1008
accuracy 0.8734622144112478
iteration 1009
accuracy 0.8717047451669596
iteration 1010
accuracy 0.859402460456942
iteration 1011
accuracy 0.8506151142355008
iteration 1012
accuracy 0.8734622144112478
iteration 1013
accuracy 0.8717047451669596
iteration 1014
accuracy 0.859402460456942
iteration 1015
accuracy 0.8506151142355008
iteration 1016
accuracy 0.8734622144112478
iteration 1017
accuracy 0.8717047451669596
iteration 1018
accuracy 0.859402460456942
iteration 1019
accuracy 0.8629173989455184
iteration 1020
accuracy 0.8629173989455184
iteration 1021
accuracy 0.859402460456942
iteration 1022
accuracy 0.875219683655536
iteration 1023
accuracy 0.875219683655536
iteration 1024
accuracy 0.8699472759226714
iteration 1025
accuracy 0.8717047451669596
iteration 1026
accuracy 0.8629173989455184
iteration 1027
accuracy 0.8506151142355008
iteration 1028
accuracy 0.8699472759226714
iteration 1029
accuracy 0.8681898066783831
iteration 1030
accuracy 0.859402

...

accuracy 0.8629173989455184
iteration 3775
accuracy 0.8629173989455184
iteration 3776
accuracy 0.8699472759226714
iteration 3777
accuracy 0.8611599297012302
iteration 3778
accuracy 0.875219683655536
iteration 3779
accuracy 0.8681898066783831
iteration 3780
accuracy 0.8611599297012302
iteration 3781
accuracy 0.8611599297012302
iteration 3782
accuracy 0.8681898066783831
iteration 3783
accuracy 0.875219683655536
iteration 3784
accuracy 0.8681898066783831
iteration 3785
accuracy 0.8681898066783831
iteration 3786
accuracy 0.875219683655536
iteration 3787
accuracy 0.8681898066783831
iteration 3788
accuracy 0.8681898066783831
iteration 3789
accuracy 0.875219683655536
iteration 3790
accuracy 0.8681898066783831
iteration 3791
accuracy 0.8611599297012302
iteration 3792
accuracy 0.8681898066783831
iteration 3793
accuracy 0.875219683655536
iteration 3794
accuracy 0.8681898066783831
iteration 3795
accuracy 0.8681898066783831
iteration 3796
accuracy 0.875219683655536
iteration 3797
accuracy 0.868189

accuracy 0.8681898066783831
iteration 4025
accuracy 0.8734622144112478
iteration 4026
accuracy 0.8734622144112478
iteration 4027
accuracy 0.8699472759226714
iteration 4028
accuracy 0.8664323374340949
iteration 4029
accuracy 0.8664323374340949
iteration 4030
accuracy 0.8664323374340949
iteration 4031
accuracy 0.8681898066783831
iteration 4032
accuracy 0.8734622144112478
iteration 4033
accuracy 0.8734622144112478
iteration 4034
accuracy 0.8699472759226714
iteration 4035
accuracy 0.8664323374340949
iteration 4036
accuracy 0.8664323374340949
iteration 4037
accuracy 0.8664323374340949
iteration 4038
accuracy 0.8664323374340949
iteration 4039
accuracy 0.8681898066783831
iteration 4040
accuracy 0.8734622144112478
iteration 4041
accuracy 0.8734622144112478
iteration 4042
accuracy 0.8699472759226714
iteration 4043
accuracy 0.8664323374340949
iteration 4044
accuracy 0.8664323374340949
iteration 4045
accuracy 0.8664323374340949
iteration 4046
accuracy 0.8664323374340949
iteration 4047
accuracy 0.

iteration 4274
accuracy 0.8664323374340949
iteration 4275
accuracy 0.8664323374340949
iteration 4276
accuracy 0.8664323374340949
iteration 4277
accuracy 0.8664323374340949
iteration 4278
accuracy 0.8664323374340949
iteration 4279
accuracy 0.8664323374340949
iteration 4280
accuracy 0.8699472759226714
iteration 4281
accuracy 0.8734622144112478
iteration 4282
accuracy 0.8681898066783831
iteration 4283
accuracy 0.8734622144112478
iteration 4284
accuracy 0.8734622144112478
iteration 4285
accuracy 0.8734622144112478
iteration 4286
accuracy 0.8734622144112478
iteration 4287
accuracy 0.8699472759226714
iteration 4288
accuracy 0.8787346221441125
iteration 4289
accuracy 0.8734622144112478
iteration 4290
accuracy 0.8664323374340949
iteration 4291
accuracy 0.8664323374340949
iteration 4292
accuracy 0.8664323374340949
iteration 4293
accuracy 0.8629173989455184
iteration 4294
accuracy 0.8664323374340949
iteration 4295
accuracy 0.8664323374340949
iteration 4296
accuracy 0.8664323374340949
iteration 4

iteration 4524
accuracy 0.8664323374340949
iteration 4525
accuracy 0.8664323374340949
iteration 4526
accuracy 0.8664323374340949
iteration 4527
accuracy 0.8629173989455184
iteration 4528
accuracy 0.8664323374340949
iteration 4529
accuracy 0.8664323374340949
iteration 4530
accuracy 0.8664323374340949
iteration 4531
accuracy 0.8664323374340949
iteration 4532
accuracy 0.8664323374340949
iteration 4533
accuracy 0.8664323374340949
iteration 4534
accuracy 0.8629173989455184
iteration 4535
accuracy 0.8664323374340949
iteration 4536
accuracy 0.8664323374340949
iteration 4537
accuracy 0.8664323374340949
iteration 4538
accuracy 0.8664323374340949
iteration 4539
accuracy 0.8664323374340949
iteration 4540
accuracy 0.8664323374340949
iteration 4541
accuracy 0.8576449912126538
iteration 4542
accuracy 0.8611599297012302
iteration 4543
accuracy 0.8611599297012302
iteration 4544
accuracy 0.8611599297012302
iteration 4545
accuracy 0.8699472759226714
iteration 4546
accuracy 0.8664323374340949
iteration 4

accuracy 0.8699472759226714
iteration 4774
accuracy 0.8734622144112478
iteration 4775
accuracy 0.8699472759226714
iteration 4776
accuracy 0.8734622144112478
iteration 4777
accuracy 0.8699472759226714
iteration 4778
accuracy 0.8734622144112478
iteration 4779
accuracy 0.8699472759226714
iteration 4780
accuracy 0.8734622144112478
iteration 4781
accuracy 0.8699472759226714
iteration 4782
accuracy 0.8787346221441125
iteration 4783
accuracy 0.8822495606326889
iteration 4784
accuracy 0.875219683655536
iteration 4785
accuracy 0.8822495606326889
iteration 4786
accuracy 0.875219683655536
iteration 4787
accuracy 0.8734622144112478
iteration 4788
accuracy 0.8804920913884007
iteration 4789
accuracy 0.8822495606326889
iteration 4790
accuracy 0.875219683655536
iteration 4791
accuracy 0.8734622144112478
iteration 4792
accuracy 0.8804920913884007
iteration 4793
accuracy 0.8734622144112478
iteration 4794
accuracy 0.8804920913884007
iteration 4795
accuracy 0.8646748681898067
iteration 4796
accuracy 0.882

In [31]:
#test for tanh function activated perceptron
#--------------------------------------
num_features = len(cells_collection[1])
#create a perceptron
tanh_p = Perceptron(num_features, num_iterations = 5000, learning_rate = 0.005, threshold = 0.5)
final_tanh_weights = tanh_p.learn_weights_tanh(cells_collection, labels)
print("final tanh-activated weights", final_tanh_weights)

iteration 0
accuracy 0.6889279437609842
iteration 1
accuracy 0.70298769771529
iteration 2
accuracy 0.718804920913884
iteration 3
accuracy 0.7750439367311073
iteration 4
accuracy 0.7680140597539543
iteration 5
accuracy 0.7855887521968365
iteration 6
accuracy 0.7873462214411248
iteration 7
accuracy 0.7996485061511424
iteration 8
accuracy 0.8137082601054482
iteration 9
accuracy 0.8260105448154658
iteration 10
accuracy 0.8154657293497364
iteration 11
accuracy 0.820738137082601
iteration 12
accuracy 0.8066783831282953
iteration 13
accuracy 0.8347978910369068
iteration 14
accuracy 0.8260105448154658
iteration 15
accuracy 0.8330404217926186
iteration 16
accuracy 0.8295254833040422
iteration 17
accuracy 0.8400702987697715
iteration 18
accuracy 0.8242530755711774
iteration 19
accuracy 0.836555360281195
iteration 20
accuracy 0.8330404217926186
iteration 21
accuracy 0.8400702987697715
iteration 22
accuracy 0.8400702987697715
iteration 23
accuracy 0.8400702987697715
iteration 24
accuracy 0.8312829

iteration 271
accuracy 0.8681898066783831
iteration 272
accuracy 0.8629173989455184
iteration 273
accuracy 0.8681898066783831
iteration 274
accuracy 0.8629173989455184
iteration 275
accuracy 0.8681898066783831
iteration 276
accuracy 0.8664323374340949
iteration 277
accuracy 0.8646748681898067
iteration 278
accuracy 0.8681898066783831
iteration 279
accuracy 0.8629173989455184
iteration 280
accuracy 0.8681898066783831
iteration 281
accuracy 0.8629173989455184
iteration 282
accuracy 0.8681898066783831
iteration 283
accuracy 0.8629173989455184
iteration 284
accuracy 0.8681898066783831
iteration 285
accuracy 0.8664323374340949
iteration 286
accuracy 0.8646748681898067
iteration 287
accuracy 0.8681898066783831
iteration 288
accuracy 0.8664323374340949
iteration 289
accuracy 0.8611599297012302
iteration 290
accuracy 0.8471001757469244
iteration 291
accuracy 0.8646748681898067
iteration 292
accuracy 0.8681898066783831
iteration 293
accuracy 0.8646748681898067
iteration 294
accuracy 0.862917398

iteration 521
accuracy 0.8681898066783831
iteration 522
accuracy 0.8681898066783831
iteration 523
accuracy 0.8681898066783831
iteration 524
accuracy 0.8681898066783831
iteration 525
accuracy 0.8681898066783831
iteration 526
accuracy 0.8681898066783831
iteration 527
accuracy 0.8717047451669596
iteration 528
accuracy 0.8664323374340949
iteration 529
accuracy 0.8699472759226714
iteration 530
accuracy 0.8664323374340949
iteration 531
accuracy 0.8646748681898067
iteration 532
accuracy 0.8681898066783831
iteration 533
accuracy 0.8699472759226714
iteration 534
accuracy 0.8681898066783831
iteration 535
accuracy 0.8664323374340949
iteration 536
accuracy 0.8699472759226714
iteration 537
accuracy 0.8699472759226714
iteration 538
accuracy 0.8664323374340949
iteration 539
accuracy 0.8681898066783831
iteration 540
accuracy 0.8664323374340949
iteration 541
accuracy 0.8664323374340949
iteration 542
accuracy 0.8681898066783831
iteration 543
accuracy 0.8646748681898067
iteration 544
accuracy 0.866432337

accuracy 0.8629173989455184
iteration 771
accuracy 0.8664323374340949
iteration 772
accuracy 0.8629173989455184
iteration 773
accuracy 0.8629173989455184
iteration 774
accuracy 0.8629173989455184
iteration 775
accuracy 0.8646748681898067
iteration 776
accuracy 0.8629173989455184
iteration 777
accuracy 0.8629173989455184
iteration 778
accuracy 0.8629173989455184
iteration 779
accuracy 0.8646748681898067
iteration 780
accuracy 0.8629173989455184
iteration 781
accuracy 0.8629173989455184
iteration 782
accuracy 0.8629173989455184
iteration 783
accuracy 0.8646748681898067
iteration 784
accuracy 0.8629173989455184
iteration 785
accuracy 0.8629173989455184
iteration 786
accuracy 0.8629173989455184
iteration 787
accuracy 0.8646748681898067
iteration 788
accuracy 0.8629173989455184
iteration 789
accuracy 0.8629173989455184
iteration 790
accuracy 0.8629173989455184
iteration 791
accuracy 0.8646748681898067
iteration 792
accuracy 0.8646748681898067
iteration 793
accuracy 0.859402460456942
iterati

accuracy 0.8629173989455184
iteration 1021
accuracy 0.859402460456942
iteration 1022
accuracy 0.875219683655536
iteration 1023
accuracy 0.875219683655536
iteration 1024
accuracy 0.8699472759226714
iteration 1025
accuracy 0.8717047451669596
iteration 1026
accuracy 0.8629173989455184
iteration 1027
accuracy 0.8506151142355008
iteration 1028
accuracy 0.8699472759226714
iteration 1029
accuracy 0.8681898066783831
iteration 1030
accuracy 0.859402460456942
iteration 1031
accuracy 0.8629173989455184
iteration 1032
accuracy 0.8629173989455184
iteration 1033
accuracy 0.859402460456942
iteration 1034
accuracy 0.875219683655536
iteration 1035
accuracy 0.875219683655536
iteration 1036
accuracy 0.8699472759226714
iteration 1037
accuracy 0.8717047451669596
iteration 1038
accuracy 0.8629173989455184
iteration 1039
accuracy 0.8506151142355008
iteration 1040
accuracy 0.8699472759226714
iteration 1041
accuracy 0.8681898066783831
iteration 1042
accuracy 0.859402460456942
iteration 1043
accuracy 0.86291739

...

accuracy 0.8681898066783831
iteration 3768
accuracy 0.8646748681898067
iteration 3769
accuracy 0.8629173989455184
iteration 3770
accuracy 0.8629173989455184
iteration 3771
accuracy 0.8629173989455184
iteration 3772
accuracy 0.8629173989455184
iteration 3773
accuracy 0.8629173989455184
iteration 3774
accuracy 0.8629173989455184
iteration 3775
accuracy 0.8629173989455184
iteration 3776
accuracy 0.8699472759226714
iteration 3777
accuracy 0.8611599297012302
iteration 3778
accuracy 0.875219683655536
iteration 3779
accuracy 0.8681898066783831
iteration 3780
accuracy 0.8611599297012302
iteration 3781
accuracy 0.8611599297012302
iteration 3782
accuracy 0.8681898066783831
iteration 3783
accuracy 0.875219683655536
iteration 3784
accuracy 0.8681898066783831
iteration 3785
accuracy 0.8681898066783831
iteration 3786
accuracy 0.875219683655536
iteration 3787
accuracy 0.8681898066783831
iteration 3788
accuracy 0.8681898066783831
iteration 3789
accuracy 0.875219683655536
iteration 3790
accuracy 0.8681

accuracy 0.8734622144112478
iteration 4018
accuracy 0.8787346221441125
iteration 4019
accuracy 0.8734622144112478
iteration 4020
accuracy 0.8787346221441125
iteration 4021
accuracy 0.8787346221441125
iteration 4022
accuracy 0.8769771528998243
iteration 4023
accuracy 0.8681898066783831
iteration 4024
accuracy 0.8681898066783831
iteration 4025
accuracy 0.8734622144112478
iteration 4026
accuracy 0.8734622144112478
iteration 4027
accuracy 0.8699472759226714
iteration 4028
accuracy 0.8664323374340949
iteration 4029
accuracy 0.8664323374340949
iteration 4030
accuracy 0.8664323374340949
iteration 4031
accuracy 0.8681898066783831
iteration 4032
accuracy 0.8734622144112478
iteration 4033
accuracy 0.8734622144112478
iteration 4034
accuracy 0.8699472759226714
iteration 4035
accuracy 0.8664323374340949
iteration 4036
accuracy 0.8664323374340949
iteration 4037
accuracy 0.8664323374340949
iteration 4038
accuracy 0.8664323374340949
iteration 4039
accuracy 0.8681898066783831
iteration 4040
accuracy 0.

iteration 4267
accuracy 0.8664323374340949
iteration 4268
accuracy 0.8664323374340949
iteration 4269
accuracy 0.8664323374340949
iteration 4270
accuracy 0.8664323374340949
iteration 4271
accuracy 0.8664323374340949
iteration 4272
accuracy 0.8664323374340949
iteration 4273
accuracy 0.8629173989455184
iteration 4274
accuracy 0.8664323374340949
iteration 4275
accuracy 0.8664323374340949
iteration 4276
accuracy 0.8664323374340949
iteration 4277
accuracy 0.8664323374340949
iteration 4278
accuracy 0.8664323374340949
iteration 4279
accuracy 0.8664323374340949
iteration 4280
accuracy 0.8699472759226714
iteration 4281
accuracy 0.8734622144112478
iteration 4282
accuracy 0.8681898066783831
iteration 4283
accuracy 0.8734622144112478
iteration 4284
accuracy 0.8734622144112478
iteration 4285
accuracy 0.8734622144112478
iteration 4286
accuracy 0.8734622144112478
iteration 4287
accuracy 0.8699472759226714
iteration 4288
accuracy 0.8787346221441125
iteration 4289
accuracy 0.8734622144112478
iteration 4

iteration 4517
accuracy 0.8664323374340949
iteration 4518
accuracy 0.8664323374340949
iteration 4519
accuracy 0.8664323374340949
iteration 4520
accuracy 0.8629173989455184
iteration 4521
accuracy 0.8664323374340949
iteration 4522
accuracy 0.8664323374340949
iteration 4523
accuracy 0.8664323374340949
iteration 4524
accuracy 0.8664323374340949
iteration 4525
accuracy 0.8664323374340949
iteration 4526
accuracy 0.8664323374340949
iteration 4527
accuracy 0.8629173989455184
iteration 4528
accuracy 0.8664323374340949
iteration 4529
accuracy 0.8664323374340949
iteration 4530
accuracy 0.8664323374340949
iteration 4531
accuracy 0.8664323374340949
iteration 4532
accuracy 0.8664323374340949
iteration 4533
accuracy 0.8664323374340949
iteration 4534
accuracy 0.8629173989455184
iteration 4535
accuracy 0.8664323374340949
iteration 4536
accuracy 0.8664323374340949
iteration 4537
accuracy 0.8664323374340949
iteration 4538
accuracy 0.8664323374340949
iteration 4539
accuracy 0.8664323374340949
iteration 4

accuracy 0.8664323374340949
iteration 4767
accuracy 0.8664323374340949
iteration 4768
accuracy 0.8664323374340949
iteration 4769
accuracy 0.8664323374340949
iteration 4770
accuracy 0.8664323374340949
iteration 4771
accuracy 0.8734622144112478
iteration 4772
accuracy 0.8734622144112478
iteration 4773
accuracy 0.8699472759226714
iteration 4774
accuracy 0.8734622144112478
iteration 4775
accuracy 0.8699472759226714
iteration 4776
accuracy 0.8734622144112478
iteration 4777
accuracy 0.8699472759226714
iteration 4778
accuracy 0.8734622144112478
iteration 4779
accuracy 0.8699472759226714
iteration 4780
accuracy 0.8734622144112478
iteration 4781
accuracy 0.8699472759226714
iteration 4782
accuracy 0.8787346221441125
iteration 4783
accuracy 0.8822495606326889
iteration 4784
accuracy 0.875219683655536
iteration 4785
accuracy 0.8822495606326889
iteration 4786
accuracy 0.875219683655536
iteration 4787
accuracy 0.8734622144112478
iteration 4788
accuracy 0.8804920913884007
iteration 4789
accuracy 0.88

In [32]:

##notes : sigmoid and tanh activation functions seem to have the vanishing gradient function problems 
#(weights stop changing once functions start to 'flatten out')